In [1]:
import os
import sqlalchemy as sa
import pandas as pd
from sample_data import stat_vectors, comp_style_vectors, map_style_vectors, roles

In [5]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

hero_stats_path = f'{BASE_DIR}\\app\sample_data\hero_stats.csv'
comfort_path = f'{BASE_DIR}\\app\sample_data\player_comfort.csv'

,HeroName,WaveClear,Engage,Peel,TeamSustain,SelfSustain,SoloLane,Fight,Macro,Artillery,Protect,Pick
0,Abathur,2.0,0.0,2.0,5.0,0.0,Maybe,4.0,8.0,1.0,7.0,2.0
1,Alarak,2.0,5.0,5.0,0.0,2.0,Maybe,5.0,3.0,2.0,4.0,9.0
2,Alexstrasza,4.0,1.0,2.0,7.0,5.0,Maybe,7.0,3.0,4.0,5.0,3.0
3,Ana,2.0,2.0,4.0,7.0,3.0,No,4.0,4.0,5.0,7.0,8.0
4,Anduin,3.0,2.0,4.0,6.0,5.0,No,4.0,3.0,3.0,7.0,7.0


In [8]:
comfort_df = pd.read_csv(comfort_path)
hero_stats_df = pd.read_csv(hero_stats_path)

hero_stats_df['SoloLaneCode'] = hero_stats_df

comfort_df.to_sql('PlayerComfort', con=engine, if_exists='replace')
hero_stats_df.to_sql('HeroStats', con=engine, if_exists='replace')

-1

In [15]:
meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

Player=meta_data.tables['Player']
Hero=meta_data.tables['Hero']
PlayerComfort=meta_data.tables['PlayerComfort']
HeroStats=meta_data.tables['HeroStats']

hero_query = sa.select(
    Hero.c.HeroId
  , Hero.c.HeroName
  , Hero.c.NewRole
)

player_query = sa.select(
  Player.c.PlayerId
  , Player.c.Battletag
  , Player.c.AccountLevel
  , Player.c.BlizzardId
  , Player.c.MasterPlayerId
)

comfort_query = sa.select(PlayerComfort)

hero_stats_query = sa.select(HeroStats)

player_df = pd.read_sql(sql = player_query, con = engine)
hero_df = pd.read_sql(sql = hero_query, con = engine)
comfort_df = pd.read_sql(sql = comfort_query, con = engine)
hero_stats_df = pd.read_sql(sql = hero_stats_query, con = engine)

hero_stats_df.head(5)

,index,HeroName,WaveClear,Engage,Peel,TeamSustain,SelfSustain,SoloLane,Fight,Macro,Artillery,Protect,Pick
0,0,Abathur,2.0,0.0,2.0,5.0,0.0,Maybe,4.0,8.0,1.0,7.0,2.0
1,1,Alarak,2.0,5.0,5.0,0.0,2.0,Maybe,5.0,3.0,2.0,4.0,9.0
2,2,Alexstrasza,4.0,1.0,2.0,7.0,5.0,Maybe,7.0,3.0,4.0,5.0,3.0
3,3,Ana,2.0,2.0,4.0,7.0,3.0,No,4.0,4.0,5.0,7.0,8.0
4,4,Anduin,3.0,2.0,4.0,6.0,5.0,No,4.0,3.0,3.0,7.0,7.0


In [17]:
hero_df['HeroRoleCode'] = hero_df.NewRole.astype('category').cat.codes
hero_stats_df['SoloLaneCode'] = hero_stats_df.SoloLane.astype('category').cat.codes

In [19]:
hero_role_df = pd.DataFrame(columns=['HeroName', 'NewRole'])
hero_role_df = hero_role_df.astype(dtype= {"HeroName":"object" })
hero_role_df.HeroName = hero_df['HeroName']
hero_role_df.NewRole = hero_df[hero_role_df['HeroName'] == hero_df['HeroName']].NewRole

for col in roles:
  hero_role_df.loc[(hero_role_df.NewRole == col), col] = 1

hero_role_df.loc[(hero_role_df.HeroName == 'Varian'), 'Tank'] = 1
hero_role_df.loc[(hero_role_df.HeroName == 'Varian'), 'Melee Assassin'] = 1

hero_role_df.fillna(0, inplace=True)
hero_role_df.head(100)

,HeroName,NewRole,Bruiser,Healer,Melee Assassin,Ranged Assassin,Support,Tank
0,Abathur,Support,0.0,0.0,0.0,0.0,1.0,0.0
1,Alarak,Melee Assassin,0.0,0.0,1.0,0.0,0.0,0.0
2,Alexstrasza,Healer,0.0,1.0,0.0,0.0,0.0,0.0
3,Ana,Healer,0.0,1.0,0.0,0.0,0.0,0.0
4,Anduin,Healer,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
85,Yrel,Bruiser,1.0,0.0,0.0,0.0,0.0,0.0
86,Zagara,Ranged Assassin,0.0,0.0,0.0,1.0,0.0,0.0
87,Zarya,Support,0.0,0.0,0.0,0.0,1.0,0.0
88,Zeratul,Melee Assassin,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
print(f'Map Count - {map_df.shape[0]}')
print(f'Hero Count - {hero_df.shape[0]}')

Map Count - 23
Hero Count - 90


In [21]:
stats_df = pd.read_csv(hero_stats_path)
players_df = pd.read_csv(comfort_path)

stats_df.set_index('HeroName', inplace=True)
players_df.set_index(['HeroName'], inplace=True)

In [22]:
df = players_df.join(stats_df, on='HeroName', how='inner')
df = pd.merge(df, hero_role_df, left_on=['HeroName', 'NewRole'], right_on=['HeroName', 'NewRole'], how='inner')

In [25]:
df.describe()

,ComfortScore,WaveClear,Engage,Peel,TeamSustain,SelfSustain,Fight,Macro,Artillery,Protect,Pick,Bruiser,Healer,Melee Assassin,Ranged Assassin,Support,Tank
count,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000
mean,5.174897,4.950617,3.543210,4.172840,1.777778,3.382716,5.111111,4.938272,4.790123,4.358025,5.506173,0.209877,0.185185,0.123457,0.308642,0.049383,0.148148
std,3.080412,1.785864,1.989971,1.994571,2.603725,2.383281,1.572968,2.118231,1.905279,1.922039,1.946438,0.407640,0.388848,0.329300,0.462409,0.216889,0.355613
min,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,2.000000,3.000000,0.000000,1.000000,4.000000,3.000000,3.000000,3.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,5.000000,4.000000,4.000000,0.000000,4.000000,5.000000,5.000000,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,6.000000,5.000000,6.000000,3.000000,5.000000,6.000000,7.000000,6.000000,6.000000,7.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,10.000000,9.000000,8.000000,9.000000,8.000000,8.000000,8.000000,9.000000,10.000000,10.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
df[['Player', 'NewRole', 'ComfortScore']].groupby(['Player', 'NewRole']).describe()

ComfortScore                                      \
                                count      mean       std  min   25%  50%   
Player   NewRole                                                            
Player 1 Bruiser                 17.0  4.588235  3.336518  0.0  2.00  4.0   
         Healer                  15.0  5.066667  2.939064  0.0  3.00  5.0   
         Melee Assassin           9.0  3.777778  1.715938  1.0  3.00  3.0   
         Ranged Assassin         25.0  5.800000  3.674235  0.0  1.00  7.0   
         Support                  4.0  6.750000  2.061553  4.0  6.25  7.0   
         Tank                    11.0  4.636364  3.471966  0.0  2.00  3.0   
Player 2 Bruiser                 17.0  5.588235  3.606571  0.0  3.00  6.0   
         Healer                  15.0  4.133333  2.614975  1.0  2.50  4.0   
         Melee Assassin           9.0  7.555556  2.403701  2.0  7.00  8.0   
         Ranged Assassin         25.0  5.920000  3.438992  0.0  2.00  7.0   
         Support                  4.0  3.500000  2.645751  1.0  1.75  3.0   
         Tank                    11.0  5.818182  3.736795  0.0  2.50  6.0   
Player 3 Bruiser                 17.0  5.294118  3.097675  0.0  3.00  5.0   
         Healer                  15.0  5.800000  3.189268  0.0  3.50  7.0   
         Melee Assassin           9.0  6.222222  2.682246  1.0  6.00  7.0   
         Ranged Assassin         25.0  5.240000  3.218178  1.0  2.00  5.0   
         Support                  4.0  5.250000  4.031129  1.0  2.50  5.0   
         Tank                    11.0  6.181818  2.891995  0.0  5.00  7.0   
Player 4 Bruiser                 17.0  5.235294  3.152963  1.0  3.00  5.0   
         Healer                  15.0  5.600000  3.224903  0.0  3.00  5.0   
         Melee Assassin           9.0  5.444444  3.205897  0.0  3.00  6.0   
         Ranged Assassin         25.0  5.120000  3.004441  0.0  3.00  5.0   
         Support                  4.0  3.250000  3.403430  0.0  1.50  2.5   
         Tank                    11.0  5.636364  2.975659  2.0  3.50  5.0   
Player 5 Bruiser                 17.0  4.470588  2.576763  1.0  3.00  4.0   
         Healer                  15.0  5.133333  2.642149  0.0  3.00  5.0   
         Melee Assassin           9.0  4.111111  3.333333  0.0  2.00  3.0   
         Ranged Assassin         25.0  4.480000  3.163858  0.0  2.00  4.0   
         Support                  4.0  4.000000  3.559026  1.0  1.00  3.5   
         Tank                    11.0  5.545455  3.297382  1.0  3.00  6.0   
Player 6 Bruiser                 17.0  5.470588  3.064263  0.0  4.00  7.0   
         Healer                  15.0  3.800000  2.704494  0.0  2.00  4.0   
         Melee Assassin           9.0  3.444444  2.068279  0.0  2.00  3.0   
         Ranged Assassin         25.0  5.000000  3.109126  0.0  3.00  4.0   
         Support                  4.0  6.750000  2.217356  4.0  5.50  7.0   
         Tank                    11.0  5.727273  2.240130  2.0  4.50  6.0   

                                      
                           75%   max  
Player   NewRole                      
Player 1 Bruiser          7.00  10.0  
         Healer           6.50  10.0  
         Melee Assassin   5.00   7.0  
         Ranged Assassin  9.00  10.0  
         Support          7.50   9.0  
         Tank             8.50   9.0  
Player 2 Bruiser          9.00  10.0  
         Healer           5.00  10.0  
         Melee Assassin   9.00  10.0  
         Ranged Assassin  9.00  10.0  
         Support          4.75   7.0  
         Tank             9.00  10.0  
Player 3 Bruiser          8.00  10.0  
         Healer           8.00  10.0  
         Melee Assassin   8.00   9.0  
         Ranged Assassin  8.00  10.0  
         Support          7.75  10.0  
         Tank             8.00  10.0  
Player 4 Bruiser          8.00  10.0  
         Healer           8.00  10.0  
         Melee Assassin   8.00  10.0  
         Ranged Assassin  8.00  10.0  
         Support          4.25   8.0  
       

In [15]:
df.groupby(['Player', 'NewRole'])['ComfortScore'].transform('mean')

0      6.750000
1      3.500000
2      5.250000
3      3.250000
4      4.000000
         ...   
481    7.555556
482    6.222222
483    5.444444
484    4.111111
485    3.444444
Name: ComfortScore, Length: 486, dtype: float64

In [13]:
df.groupby(['Player', 'NewRole'])['ComfortScore'].nlargest(2)

Player    NewRole             
Player 1  Bruiser          174    10
                           396    10
          Healer           12     10
                           348    10
          Melee Assassin   210     7
                                  ..
Player 6  Ranged Assassin  293    10
          Support          245     9
                           5       8
          Tank             125    10
                           113     8
Name: ComfortScore, Length: 72, dtype: int64